# Deep Q-Learning 

For this assignment we will implement the Deep Q-Learning algorithm with Experience Replay as described in breakthrough paper __"Playing Atari with Deep Reinforcement Learning"__. We will train an agent to play the famous game of __Breakout__.

In [1]:
import sys
import gym
import torch
import pylab
import random
import numpy as np
from collections import deque
from datetime import datetime
from copy import deepcopy
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from utils import *
from agent import *
from model import *
from config import *
%matplotlib inline
%load_ext autoreload
%autoreload 2

## Understanding the environment

In the following cell, we initialise our game of __Breakout__ and you can see how the environment looks like. For further documentation of the of the environment refer to https://gym.openai.com/envs. 

In [2]:
env = gym.make('BreakoutDeterministic-v4')
env.render()

True

In [3]:
number_lives = find_max_lifes(env)
state_size = env.observation_space.shape
action_size = 3
rewards, episodes = [], []

## Creating a DQN Agent

Here we create a DQN Agent. This agent is defined in the __agent.py__. The corresponding neural network is defined in the __model.py__. 

__Evaluation Reward__ : The average reward received in the past 100 episodes/games.

__Frame__ : Number of frames processed in total.

__Memory Size__ : The current size of the replay memory.

In [4]:
agent = Agent(action_size)
evaluation_reward = deque(maxlen=evaluation_reward_length)
frame = 0
memory_size = 0

### Main Training Loop

In [ ]:
for e in range(EPISODES):
    done = False
    score = 0

    history = np.zeros([5, 84, 84], dtype=np.uint8)
    step = 0
    d = False
    state = env.reset()
    life = number_lives

    get_init_state(history, state)

    while not done:
        step += 1
        frame += 1
        if render_breakout:
            env.render()

        # Select and perform an action
        action = agent.get_action(np.float32(history[:4, :, :]) / 255.)

        
        next_state, reward, done, info = env.step(action + 1)

        frame_next_state = get_frame(next_state)
        history[4, :, :] = frame_next_state
        terminal_state = check_live(life, info['ale.lives'])

        life = info['ale.lives']
        r = np.clip(reward, -1, 1)

        # Store the transition in memory 
        agent.memory.push(deepcopy(frame_next_state), action, r, terminal_state)
        # Start training after random sample generation
        if(frame >= train_frame):
            agent.train_policy_net(frame)
            # Update the target network
            if(frame % Update_target_network_frequency)== 0:
                agent.update_target_net()
        score += reward
        history[:4, :, :] = history[1:, :, :]

        if frame % 50000 == 0:
            print('now time : ', datetime.now())
            rewards.append(np.mean(evaluation_reward))
            episodes.append(e)
            pylab.plot(episodes, rewards, 'b')
            pylab.savefig("./save_graph/breakout_dqn.png")

        if done:
            evaluation_reward.append(score)
            # every episode, plot the play time
            print("episode:", e, "  score:", score, "  memory length:",
                  len(agent.memory), "  epsilon:", agent.epsilon, "   steps:", step,
                  "    evaluation reward:", np.mean(evaluation_reward))

            # if the mean of scores of last 10 episode is bigger than 400
            # stop training
            if np.mean(evaluation_reward) > 400:
                torch.save(agent.model, "./save_model/breakout_dqn")
                sys.exit()

episode: 0   score: 2.0   memory length: 220   epsilon: 1.0    steps: 220     evaluation reward: 2.0
episode: 1   score: 1.0   memory length: 396   epsilon: 1.0    steps: 176     evaluation reward: 1.5
episode: 2   score: 0.0   memory length: 528   epsilon: 1.0    steps: 132     evaluation reward: 1.0
episode: 3   score: 2.0   memory length: 739   epsilon: 1.0    steps: 211     evaluation reward: 1.25
episode: 4   score: 0.0   memory length: 875   epsilon: 1.0    steps: 136     evaluation reward: 1.0
episode: 5   score: 2.0   memory length: 1067   epsilon: 1.0    steps: 192     evaluation reward: 1.1666666666666667
episode: 6   score: 1.0   memory length: 1224   epsilon: 1.0    steps: 157     evaluation reward: 1.1428571428571428
episode: 7   score: 2.0   memory length: 1409   epsilon: 1.0    steps: 185     evaluation reward: 1.25
episode: 8   score: 0.0   memory length: 1534   epsilon: 1.0    steps: 125     evaluation reward: 1.1111111111111112
episode: 9   score: 0.0   memory length:

episode: 73   score: 0.0   memory length: 13703   epsilon: 1.0    steps: 133     evaluation reward: 1.4054054054054055
episode: 74   score: 1.0   memory length: 13877   epsilon: 1.0    steps: 174     evaluation reward: 1.4
episode: 75   score: 0.0   memory length: 14008   epsilon: 1.0    steps: 131     evaluation reward: 1.381578947368421
episode: 76   score: 0.0   memory length: 14133   epsilon: 1.0    steps: 125     evaluation reward: 1.3636363636363635
episode: 77   score: 0.0   memory length: 14262   epsilon: 1.0    steps: 129     evaluation reward: 1.3461538461538463
episode: 78   score: 0.0   memory length: 14392   epsilon: 1.0    steps: 130     evaluation reward: 1.3291139240506329
episode: 79   score: 2.0   memory length: 14596   epsilon: 1.0    steps: 204     evaluation reward: 1.3375
episode: 80   score: 0.0   memory length: 14732   epsilon: 1.0    steps: 136     evaluation reward: 1.3209876543209877
episode: 81   score: 1.0   memory length: 14895   epsilon: 1.0    steps: 163

episode: 148   score: 0.0   memory length: 27472   epsilon: 1.0    steps: 128     evaluation reward: 1.35
episode: 149   score: 0.0   memory length: 27612   epsilon: 1.0    steps: 140     evaluation reward: 1.33
episode: 150   score: 3.0   memory length: 27850   epsilon: 1.0    steps: 238     evaluation reward: 1.31
episode: 151   score: 0.0   memory length: 27977   epsilon: 1.0    steps: 127     evaluation reward: 1.31
episode: 152   score: 1.0   memory length: 28136   epsilon: 1.0    steps: 159     evaluation reward: 1.3
episode: 153   score: 0.0   memory length: 28266   epsilon: 1.0    steps: 130     evaluation reward: 1.28
episode: 154   score: 0.0   memory length: 28395   epsilon: 1.0    steps: 129     evaluation reward: 1.28
episode: 155   score: 2.0   memory length: 28587   epsilon: 1.0    steps: 192     evaluation reward: 1.29
episode: 156   score: 2.0   memory length: 28805   epsilon: 1.0    steps: 218     evaluation reward: 1.31
episode: 157   score: 2.0   memory length: 2899

episode: 226   score: 1.0   memory length: 40453   epsilon: 1.0    steps: 158     evaluation reward: 1.04
episode: 227   score: 2.0   memory length: 40675   epsilon: 1.0    steps: 222     evaluation reward: 1.06
episode: 228   score: 2.0   memory length: 40889   epsilon: 1.0    steps: 214     evaluation reward: 1.06
episode: 229   score: 1.0   memory length: 41049   epsilon: 1.0    steps: 160     evaluation reward: 1.05
episode: 230   score: 2.0   memory length: 41253   epsilon: 1.0    steps: 204     evaluation reward: 1.06
episode: 231   score: 2.0   memory length: 41460   epsilon: 1.0    steps: 207     evaluation reward: 1.08
episode: 232   score: 1.0   memory length: 41633   epsilon: 1.0    steps: 173     evaluation reward: 1.09
episode: 233   score: 2.0   memory length: 41833   epsilon: 1.0    steps: 200     evaluation reward: 1.09
episode: 234   score: 0.0   memory length: 41969   epsilon: 1.0    steps: 136     evaluation reward: 1.08
episode: 235   score: 1.0   memory length: 421

episode: 301   score: 3.0   memory length: 53716   epsilon: 0.9926403400001598    steps: 254     evaluation reward: 1.08
episode: 302   score: 1.0   memory length: 53873   epsilon: 0.9923294800001665    steps: 157     evaluation reward: 1.08
episode: 303   score: 0.0   memory length: 54013   epsilon: 0.9920522800001725    steps: 140     evaluation reward: 1.06
episode: 304   score: 0.0   memory length: 54145   epsilon: 0.9917909200001782    steps: 132     evaluation reward: 1.06
episode: 305   score: 0.0   memory length: 54277   epsilon: 0.9915295600001839    steps: 132     evaluation reward: 1.03
episode: 306   score: 1.0   memory length: 54456   epsilon: 0.9911751400001916    steps: 179     evaluation reward: 1.03
episode: 307   score: 0.0   memory length: 54584   epsilon: 0.9909217000001971    steps: 128     evaluation reward: 1.02
episode: 308   score: 2.0   memory length: 54824   epsilon: 0.9904465000002074    steps: 240     evaluation reward: 1.04
episode: 309   score: 0.0   memo

...

episode: 2721   score: 0.0   memory length: 490071   epsilon: 0.12865744000850504    steps: 122     evaluation reward: 0.96
episode: 2722   score: 0.0   memory length: 490193   epsilon: 0.1284158800085035    steps: 122     evaluation reward: 0.96
episode: 2723   score: 2.0   memory length: 490390   epsilon: 0.12802582000850105    steps: 197     evaluation reward: 0.96
episode: 2724   score: 0.0   memory length: 490512   epsilon: 0.12778426000849952    steps: 122     evaluation reward: 0.96
episode: 2725   score: 1.0   memory length: 490662   epsilon: 0.12748726000849764    steps: 150     evaluation reward: 0.95
episode: 2726   score: 0.0   memory length: 490784   epsilon: 0.1272457000084961    steps: 122     evaluation reward: 0.95
episode: 2727   score: 2.0   memory length: 490981   epsilon: 0.12685564000849364    steps: 197     evaluation reward: 0.97
episode: 2728   score: 0.0   memory length: 491103   epsilon: 0.1266140800084921    steps: 122     evaluation reward: 0.97
episode: 27

episode: 2787   score: 1.0   memory length: 500676   epsilon: 0.10765954000849373    steps: 150     evaluation reward: 1.15
episode: 2788   score: 2.0   memory length: 500873   epsilon: 0.107269480008494    steps: 197     evaluation reward: 1.14
episode: 2789   score: 0.0   memory length: 500996   epsilon: 0.10702594000849416    steps: 123     evaluation reward: 1.13
episode: 2790   score: 0.0   memory length: 501118   epsilon: 0.10678438000849433    steps: 122     evaluation reward: 1.13
episode: 2791   score: 0.0   memory length: 501240   epsilon: 0.1065428200084945    steps: 122     evaluation reward: 1.11
episode: 2792   score: 0.0   memory length: 501362   epsilon: 0.10630126000849466    steps: 122     evaluation reward: 1.09
episode: 2793   score: 3.0   memory length: 501608   epsilon: 0.10581418000849499    steps: 246     evaluation reward: 1.11
episode: 2794   score: 0.0   memory length: 501730   epsilon: 0.10557262000849515    steps: 122     evaluation reward: 1.1
episode: 279

episode: 2854   score: 1.0   memory length: 511248   epsilon: 0.08672698000850801    steps: 150     evaluation reward: 1.07
episode: 2855   score: 4.0   memory length: 511504   epsilon: 0.08622010000850835    steps: 256     evaluation reward: 1.1
episode: 2856   score: 0.0   memory length: 511627   epsilon: 0.08597656000850852    steps: 123     evaluation reward: 1.1
episode: 2857   score: 0.0   memory length: 511749   epsilon: 0.08573500000850869    steps: 122     evaluation reward: 1.1
episode: 2858   score: 0.0   memory length: 511871   epsilon: 0.08549344000850885    steps: 122     evaluation reward: 1.08
episode: 2859   score: 2.0   memory length: 512054   epsilon: 0.0851311000085091    steps: 183     evaluation reward: 1.09
episode: 2860   score: 0.0   memory length: 512176   epsilon: 0.08488954000850926    steps: 122     evaluation reward: 1.07
episode: 2861   score: 0.0   memory length: 512298   epsilon: 0.08464798000850943    steps: 122     evaluation reward: 1.07
episode: 286

episode: 2921   score: 0.0   memory length: 521089   epsilon: 0.0672418000085213    steps: 122     evaluation reward: 0.82
episode: 2922   score: 1.0   memory length: 521239   epsilon: 0.0669448000085215    steps: 150     evaluation reward: 0.8
episode: 2923   score: 0.0   memory length: 521361   epsilon: 0.06670324000852167    steps: 122     evaluation reward: 0.8
episode: 2924   score: 0.0   memory length: 521483   epsilon: 0.06646168000852183    steps: 122     evaluation reward: 0.8
episode: 2925   score: 0.0   memory length: 521606   epsilon: 0.066218140008522    steps: 123     evaluation reward: 0.78
episode: 2926   score: 0.0   memory length: 521728   epsilon: 0.06597658000852216    steps: 122     evaluation reward: 0.78
episode: 2927   score: 0.0   memory length: 521851   epsilon: 0.06573304000852233    steps: 123     evaluation reward: 0.78
episode: 2928   score: 2.0   memory length: 522034   epsilon: 0.06537070000852258    steps: 183     evaluation reward: 0.8
episode: 2929   

episode: 2988   score: 0.0   memory length: 530775   epsilon: 0.04806352000853438    steps: 122     evaluation reward: 0.72
episode: 2989   score: 0.0   memory length: 530897   epsilon: 0.047821960008534545    steps: 122     evaluation reward: 0.72
episode: 2990   score: 1.0   memory length: 531066   epsilon: 0.04748734000853477    steps: 169     evaluation reward: 0.71
episode: 2991   score: 0.0   memory length: 531188   epsilon: 0.04724578000853494    steps: 122     evaluation reward: 0.7
episode: 2992   score: 0.0   memory length: 531311   epsilon: 0.047002240008535104    steps: 123     evaluation reward: 0.7
episode: 2993   score: 0.0   memory length: 531433   epsilon: 0.04676068000853527    steps: 122     evaluation reward: 0.68
episode: 2994   score: 0.0   memory length: 531555   epsilon: 0.04651912000853543    steps: 122     evaluation reward: 0.68
episode: 2995   score: 1.0   memory length: 531705   epsilon: 0.046222120008535636    steps: 150     evaluation reward: 0.69
episode